# Ukraine Energy Dashboard - Data Inspection


In [1]:
# Import necessary libraries
import geopandas as gpd

gdf = gpd.read_file("/workspaces/ukr_energy_dash/assets/data/power_stations_with_oblasts.geojson")

In [8]:
# sort by oblast, then power, power:source
gdf = gdf.sort_values(by=["oblast_name_en", "name", "power", "plant:method"], ascending=[True, True, True, True])

In [10]:
gdf.head()

,osm_id,osm_type,power,substation,name,station_name_en,operator,operator:en,barrier,voltage,landuse,plant:method,plant:output:electricity,plant:source,oblast_name_en,gppd_overlap,geometry
527,839716761,way,plant,None,Вороненська ГЕС,None,None,None,None,None,industrial,None,yes,hydro,Cherkasy,False,"POLYGON ((30.23471 49.14875, 30.23474 49.14865..."
274,360746175,way,plant,None,Закинута гідроелектростанція,None,None,None,None,None,None,None,yes,hydro,Cherkasy,False,"POLYGON ((30.39681 49.09151, 30.39702 49.09146..."
545,865045756,way,plant,None,Звенигородська ГЕС,None,"ЗОВНІШНЬОЕКОНОМІЧНА АСОЦІАЦІЯ ""НОВОСВІТ""",None,None,None,industrial,water-storage,0.4 MW,hydro,Cherkasy,False,"POLYGON ((30.94181 49.09441, 30.94192 49.09439..."
12,73199860,way,plant,None,Канівська ГЕС,None,None,None,None,None,industrial,run-of-the-river,472 MW,hydro,Cherkasy,True,"POLYGON ((31.45975 49.76252, 31.45949 49.76242..."
572,909962483,way,plant,None,Корсунь-Шевченківська ГЕС,None,None,None,None,None,industrial,None,1.8 MW,hydro,Cherkasy,False,"POLYGON ((31.26293 49.41092, 31.26287 49.41129..."


In [12]:
# counts per power source
gdf["plant:source"].value_counts()

plant:source
solar             380
gas               136
hydro              87
coal               21
wind               19
nuclear             5
biogas              4
biomass             3
gas;oil             3
gas;coal            2
diesel              1
coal;gas;oil        1
coal;oil            1
waste               1
oil;gas             1
oil;gas;diesel      1
gas;mazut           1
wood                1
coal;gas            1
Name: count, dtype: int64

In [10]:
gdf.columns

Index(['osm_id', 'osm_type', 'power', 'substation', 'name', 'station_name_en',
       'operator', 'operator:en', 'barrier', 'voltage', 'landuse',
       'plant:method', 'plant:output:electricity', 'plant:source',
       'oblast_name_en', 'gppd_overlap', 'geometry'],
      dtype='object')

In [ ]:
gdf[gdf["plant:output:electricity"] == "351 MW"]

,osm_id,osm_type,power,substation,name,station_name_en,operator,operator:en,barrier,voltage,landuse,plant:method,plant:output:electricity,plant:source,oblast_name_en,gppd_overlap,geometry
943,7317657,relation,plant,None,колишня Каховська ГЕС імені П. С. Непорожнього,ex Kakhovskaya HPP named after P. S. Neporozhny,ВАТ «Укргідроенерго»,None,None,None,industrial,None,351 MW,hydro,Kherson,True,"MULTIPOLYGON (((33.36813 46.78048, 33.37141 46..."
